In [1]:
import pandas as pd
df = pd.read_csv('../../readwise-data.csv')
def passage_text(record):
    return f'You highlighted the following passage on {record['Highlighted at']}: << {record['Highlight']} >>. It comes from \'{record['Book Title']}\' by {record['Book Author']}'
df['passage_text'] = df.apply(passage_text, axis=1)
df = df#.sample(3000)
data = df.to_dict('records')
df

,Highlight,Book Title,Book Author,Amazon Book ID,Note,Color,Tags,Location Type,Location,Highlighted at,Document tags,passage_text
0,When interacting with others about a new or co...,"Gather, decide, execute",James Stanier,NaN,NaN,NaN,NaN,NaN,0.0,2025-02-01 15:37:10+00:00,NaN,You highlighted the following passage on 2025-...
1,I try to fight against a natural reduction in ...,"Gather, decide, execute",James Stanier,NaN,NaN,NaN,NaN,order,1.0,2025-02-01 15:37:45+00:00,NaN,You highlighted the following passage on 2025-...
2,a skill that I've been practicing is trying no...,"Gather, decide, execute",James Stanier,NaN,NaN,NaN,NaN,order,2.0,2025-02-01 15:39:33+00:00,NaN,You highlighted the following passage on 2025-...
3,Even if you aren't making an explicit decision...,"Gather, decide, execute",James Stanier,NaN,NaN,NaN,NaN,order,3.0,2025-02-01 15:41:39+00:00,NaN,You highlighted the following passage on 2025-...
4,Socrates discovered you can inquire into deepe...,Argue Your Way to a Fuller Life,Steve Paulson,NaN,NaN,NaN,NaN,NaN,0.0,2025-01-23 21:41:56+00:00,"relationships,philosophy,argumentation",You highlighted the following passage on 2025-...
...,...,...,...,...,...,...,...,...,...,...,...,...
6915,Medios y analistas seguimos demasiado obsesion...,Parar para avanzar (Spanish Edition),Sandra Borda,B08D2RSLNR,NaN,yellow,NaN,location,1067.0,2020-08-26 04:56:00+00:00,NaN,You highlighted the following passage on 2020-...
6916,No estamos familiarizados con los movimientos ...,Parar para avanzar (Spanish Edition),Sandra Borda,B08D2RSLNR,NaN,yellow,NaN,location,1074.0,2020-08-26 04:56:00+00:00,NaN,You highlighted the following passage on 2020-...
6917,Luego las organizaciones estudiantiles son esc...,Parar para avanzar (Spanish Edition),Sandra Borda,B08D2RSLNR,NaN,yellow,NaN,location,1103.0,2020-08-26 04:56:00+00:00,NaN,You highlighted the following passage on 2020-...
6918,Hemos descubierto que la democracia no solamen...,Parar para avanzar (Spanish Edition),Sandra Borda,B08D2RSLNR,NaN,yellow,NaN,location,1114.0,2020-08-26 04:56:00+00:00,NaN,You highlighted the following passage on 2020-...


In [2]:
from qdrant_client import models, QdrantClient
from sentence_transformers import SentenceTransformer

In [3]:
encoder = SentenceTransformer('all-MiniLM-L6-v2') # Model to create embeddings

In [4]:
# create the vector database client
qdrant = QdrantClient(":memory:") # Create in-memory Qdrant instance

In [5]:
# Create collection to store books
qdrant.recreate_collection(
    collection_name="highlights",
    vectors_config=models.VectorParams(
        size=encoder.get_sentence_embedding_dimension(), # Vector size is defined by used model
        distance=models.Distance.COSINE
    )
)

/var/folders/6g/d16gbw5x5z9fk0rmvxw5z5q80000gn/T/ipykernel_5768/4189301970.py:2: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdrant.recreate_collection(


True

In [6]:
%%time
# vectorize!
qdrant.upload_points(
    collection_name="highlights",
    points=[
        models.PointStruct(
            id=idx,
            vector=encoder.encode(doc["passage_text"]).tolist(),
            payload=doc
        ) for idx, doc in enumerate(data) # data is the variable holding all the wines
    ]
)

CPU times: user 2min 37s, sys: 15.4 s, total: 2min 53s
Wall time: 2min 52s


In [13]:
from openai import OpenAI
import os
from IPython.display import display, Markdown

user_prompt = 'Summarize techniques that can be used to persuade someone on changing their mind and making changes'

hits = qdrant.search(
    collection_name="highlights",
    query_vector=encoder.encode(user_prompt).tolist(),
    limit=25
)
for hit in hits:
  print(f'{hit.payload['Highlight']} at {hit.payload['Book Title']}', "score:", hit.score)
search_results = [hit.payload for hit in hits]

client = OpenAI(
    api_key = os.environ["OPENAI_API_KEY_RAG"]
)
completion = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "You are chatbot, you have a list of Readwise highlights from the user and your job is to respond to prompts related to those highlights. When answering, include the sources."},
        {"role": "user", "content": user_prompt},
        {"role": "assistant", "content": str(search_results)}
    ]
)
display(Markdown((completion.choices[0].message.content)))

The only way they are going to change their mind is by changing their own mind-by talking themselves through their own thinking, by processing things they've never thought about before, things from their own life that are going to help them see things differently." at How Minds Change: The Surprising Science of Belief, Opinion, and Persuasion score: 0.5298886995396246
If they’ve clearly articulated a logical set of objections, and they don’t appear to be hiding ulterior motives, approach them with a cognitive conversation. This is especially useful when the detractor is known to have a no-nonsense attitude and can easily set aside emotions in their decision-making process at How to (Actually) Change Someone’s Mind score: 0.48276613250574807
The leaders who were most successful in overcoming others’ skepticism were those who diagnosed the root of the fundamental disagreement before trying to persuade. They first asked themselves, “What’s driving my detractor’s resistance?” These leaders

To effectively persuade someone to change their mind, various techniques and principles can be utilized:

1. **Self-Reflection and Personal Processing**: Individuals are more likely to change their minds when they process their thoughts and life experiences, leading them to new perspectives (David McRaney, "How Minds Change: The Surprising Science of Belief, Opinion, and Persuasion").

2. **Logical and Cognitive Conversations**: Engage in a cognitive conversation when the person's objections are clear and logical, especially if they can set aside emotions (Laura Huang, Ryan Yu, "How to (Actually) Change Someone’s Mind").

3. **Understanding the Root of Disagreement**: Successful leaders diagnose the root of disagreements before trying to persuade, asking what drives resistance and identifying points that provoke emotional reactions (Laura Huang, Ryan Yu).

4. **Building Rapport and Understanding**: Rather than immediately trying to convince, invest time in understanding and building rapport, focusing not on arguments but on understanding perspectives (Laura Huang, Ryan Yu).

5. **Central vs. Peripheral Route**: Attitude change via the central route takes effort but produces enduring results, while the peripheral route is quicker but less lasting (David McRaney).

6. **Sound Arguments and Presentation**: A successful persuasive conversation requires both sound arguments and effective presentation (Laura Huang, Ryan Yu).

7. **Emotional Detachment**: Avoid introducing emotions which could suggest a lack of common ground (Laura Huang, Ryan Yu).

8. **Storytelling**: Stories engage without directly challenging autonomy or identity, making them powerful persuasive tools (David McRaney).

9. **Technique Rebuttal**: Encourages self-reflection by asking individuals to reconsider their reasoning process and the soundness of their conclusions (David McRaney).

10. **Motivational Interviewing**: Addresses readiness to change, recognizing factors such as feeling threatened or stuck in a rationalization loop (David McRaney).

11. **Third-party Credibility**: Bringing in a credible colleague can help disentangle personal biases from the arguments, leading to objective evaluation (Laura Huang, Ryan Yu).

12. **Maintaining Relationship Integrity**: Focus on demonstrating that you are not an outsider, strengthening the relationship throughout the persuasion attempt (David McRaney).

13. **Avoiding Defensive Triggers**: Techniques are less effective if the person feels under attack; thus, it’s essential to avoid triggering defensive reflexes (David McRaney).

These techniques, supported by evidence and methodologies like "deep canvassing" and "motivational interviewing," aim to create conditions where individuals are open to exploring and potentially revising their beliefs and opinions.